In [ ]:
!pip install gradio --quiet
!pip install transformers
!pip install PyPDF2
import gradio as gr
from PyPDF2 import PdfReader
from transformers import BartTokenizer, BartForConditionalGeneration

# Load pre-trained BART model and tokenizer
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Function to generate a summary using the BART model
def summarize_bart(text):
    # Tokenize input text
    inputs = bart_tokenizer([text], max_length=1024, return_tensors='pt', truncation=True, padding=True)

    # Generate summary ids
    summary_ids = bart_model.generate(inputs['input_ids'],
                                      max_length=300, min_length=100,
                                      length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary and return
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary

def summarize_pdf(pdf_file):
    # Extract text from PDF
    reader = PdfReader(pdf_file.name)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    if not text.strip():
        return "No text found in the PDF!"

    # Summarize the extracted text (truncate to 2000 characters for efficiency)
    truncated_text = text[:2000]
    return summarize_bart(truncated_text)

def summarize(input_method, text_input=None, pdf_file=None):
    if input_method == "Text" and text_input:
        return summarize_bart(text_input)
    elif input_method == "PDF" and pdf_file:
        return summarize_pdf(pdf_file)
    else:
        return "Please provide valid input for the selected method."

css= """
h1{
  margin-top: 2rem;
  font-size: 2rem;
  text-align: center;
}
"""

# Define the Gradio interface
with gr.Blocks(css=css) as demo:
    gr.Markdown("# Text Summarization App")
    gr.Markdown("Choose an input method to provide the content for summarization.")

    input_method = gr.Radio(["Text", "PDF"], label="Input Method", value="Text")
    text_input = gr.Textbox(label="Input Text", placeholder="Enter your text here...", visible=True)
    pdf_file = gr.File(label="Upload PDF", file_types=[".pdf"], visible=False)

    result = gr.Textbox(label="Summary", interactive=False)

    def toggle_inputs(method):
        if method == "Text":
            return gr.update(visible=True), gr.update(visible=False)
        elif method == "PDF":
            return gr.update(visible=False), gr.update(visible=True)

    input_method.change(toggle_inputs, inputs=[input_method], outputs=[text_input, pdf_file])
    summarize_button = gr.Button("Summarize")
    summarize_button.click(
        summarize,
        inputs=[input_method, text_input, pdf_file],
        outputs=result
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://952d57147b97ffebf4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
